In [113]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import Dataset, Dataloader
import time
import sys
import pytorch_model_summary
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image

print(f'torch : {torch.__version__}')
print(f'python : {sys.version}')

ImportError: cannot import name 'Dataloader' from 'torch.utils.data' (/opt/conda/lib/python3.8/site-packages/torch/utils/data/__init__.py)

In [184]:

class ImageData(Dataset):
    def __init__(self,path,transform=None,target_transform=None):
        self.path = path
        self.files =[]
        self.labels = []
        for i in os.listdir(path):
            if i[:-3]+'png' in os.listdir(path+'/label'):
                self.files.append(i)
                self.labels.append(i[:-3]+'png')
        self.transform = transform
        self.target_transform = target_transform
        self.to_tensor = transforms.ToTensor()
        
    def __getitem__(self,idx):
        img = Image.open(self.path+'/'+self.files[idx])
        label = Image.open(self.path+'/label/'+self.labels[idx])
        #img = np.array(img)[:,:256]
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            label = self.target_transform(label)
        return img, label

    def __len__(self):
        return len(self.files)


In [185]:
transform = transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ]
)
label_transform = transforms.Compose(
    [
        transforms.Resize((224,224)),
        transforms.ToTensor()
    ]
)

batch_size = 64
train_path = './data/train'
train_set = ImageData(train_path,transform,label_transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size= batch_size,shuffle=True)
val_path = './data/val'
val_set = ImageData(val_path,transform,label_transform)
val_loader = torch.utils.data.DataLoader(val_set, batch_size= batch_size,shuffle=True)

In [186]:
a,b = train_set[0]
b =np.array(b)
print()

(1, 224, 224)


In [ ]:
class FCN32(nn.Module):
    def __init__(self,num_class):
        super(FCN32, self).__init__()
        self.